[GoogleTest]: https://github.com/google/googletest
# スパイ(Spying)とモック(Mocking)

[GoogleTest][]を使って、
[テスト駆動開発による組み込みプログラミング
――C言語とオブジェクト指向で学ぶアジャイルな設計](https://www.oreilly.co.jp/books/9784873116143/)
を実践に応用しました。

テストにおけるスパイとモックとは、以下のような役割のものと解釈しました。  
※個人的な解釈ですので、厳密には間違っている可能性があります。

### スパイ(Spying)
* テスト対象のオブジェクト(Object Under Test:OUT)に渡されたパラメタを記録して、意図した値・意図した頻度で
パラメタが渡されているかを記録します。
* テスト対象のオブジェクト(OUT)内部で保持するデータ構造が意図した通りに操作されているか、
意図したデータが記録されているかを確認します。

![Spying](images/Spying.png)

### モック(Mocking)
* テスト対象のオブジェクト(OUT)が、他のオブジェクトに依存している場合、
そのオブジェクトの代用品として動作します。
* テスト対象のオブジェクト(OUT)が、他のオブジェクトを意図した通りに呼び出しているか、
他のオブジェクトから出力された結果を意図通りに操作しているかを確認します。

![Mocking](images/Mocking.png)


# Spying

Spy の利用においては、Spy によるテストを踏まえて、テスト対象コードの実装方法についても考慮が必要です。
以下にテスト対象コードの実装方法について注意点を記します。

#### OUT内部データ構造へのアクセス
* `*.c` に構造体定義を置かないこと。 
* private な構造体は `*_private.h` などに定義し、
本実装では `#include` の呼び出し元を制限しておくこと。

こうすることで、Spyコードが内部データに対してアクセスできます。

#### OUT内部関数の Wrapping
* 多段 spy が出来るように、OUT内のプライベート関数は、関数テーブルからの呼び出しにすること

こうすることで、OUT内のプライベート関数も Spy 関数による置き換えができるようになり、
OUTの挙動を確認できます。


# テストを考慮した実装例

以下のように、メソッドと内部データを保持するための構造体を定義していたとします。
このような定義ファイルは "*.c" では定義せず、"*.h"として別ファイルに置きます。

```C:hoge_private.h
typedef struct hoge_struct {
    void (*self_destroy)(struct hoge_struct *self);

    void (*inc_x)(struct hoge_struct *self, int x);
    void (*inc_10)(struct hoge_struct *self);
    bool (*ge_x)(struct hoge_struct *self, int x);
    
    int total;
} *hoge_ptr;
```


# テストを考慮した実装例(cont'd)

この hoge モジュールのメソッドを以下のように実装します。

```C:hoge.c
// private definitions
#include "hoge_private.h"

// prototypes of private functions
static void self_destroy(struct hoge_struct *self);
static void inc_x(struct hoge_struct *self, int x);
static void inc_10(struct hoge_struct *self);
static bool ge_x(struct hoge_struct *self, int x);

// public functions
void *
hoge_new(void)
{
    void *h;
    h = osal_calloc(1, sizeof(struct hoge_struct));
    if (!h) {
        // error message
        return NULL;
    }
    hoge_ptr hoge = (hoge_ptr)h;
    hoge->self_destroy = self_destroy;
    hoge->inc_x = inc_x;
    hoge->inc_10 = inc_10;
    hoge->ge_x = ge_x;
    hoge->total = 0;
    return hoge;
}

// private functions

static void
self_destroy(struct hoge_struct *self)
{
    if (!self)
        return;
    osal_free(self);
}

static void
inc_x(struct hoge_struct *self, int x)
{
    self->total += x;
}

static void
inc_10(struct hoge_struct *self)
{
    self->total += 10;
}

static bool
ge_x(struct hoge_struct *self, int x)
{
    return (self->total >= x) ? true : false;
}
```


# テストを考慮した実装例(cont'd)

さらに hoge モジュールのメソッドを他から呼び出させるように、
hoge モジュール用APIを定義します。

```C:hoge.c
void
hoge_destroy(void *self)
{
    hoge_ptr hoge = (hoge_ptr)self;
    if (!hoge->self_destroy)
        return;
    (*hoge->self_destroy)(hoge);
}

void
hoge_inc(void *self, int x)
{
    hoge_ptr hoge = (hoge_ptr)self;
    if (!hoge->inc_x)
        return;
    (*hoge->inc_x)(hoge, x);
}

void
hoge_inc10(void *self)
{
    hoge_ptr hoge = (hoge_ptr)self;
    if (!hoge->inc_10)
        return;
    (*hoge->inc_10)(hoge);
}

bool
hoge_ge(void *self, int x)
{
    hoge_ptr hoge = (hoge_ptr)self;
    if (!hoge->ge_x)
        return;
    return (*hoge->ge_x)(hoge, x);
}
```

他モジュールから呼び出せるように、
ヘッダファイルに関数を宣言します。

```C:hoge.h
void *hoge_new(void);
void hoge_destroy(void *h);

void hoge_inc(void *self, int x);
void hoge_inc10(void *self);
bool hoge_ge(void *self, int x);
```


# Spy の実装

この例の Spy はモジュール hoge の関数呼び出しの回数をカウントして、
その呼び出し回数を取得することができます。

そこで、spy用に spy_hoge_struct を定義して、
original の hoge_struct へのアクセスを
spy 用の hoge_struct を経由させるようにします。

```C:spy_hoge.c
// OUT の private データ構造体(struct hoge_struct)
#include "hoge_private.h"

typedef struct spy_hoge_struct {
    struct hoge_struct spy; ///< spy用の hoge_struct
    struct hoge_struct orig;///< original の hoge_struct
    
    int call_count[3/*inc,inc10,ge*/]; ///< 呼び出し履歴記録
} *spy_hoge;
```

# Spy の実装(cont'd)

spy用の hoge_struct のコンストラクタを以下のように、
realloc()でメモリ領域を拡張して、
先頭の hoge メソッドを spy メソッドに置き換えます。

```C:spy_hoge.c
void *
spy_hoge_new(void)
{
    void *orig;
    orig = hoge_new();
    if (!orig) {
        return NULL;
    }
    spy_hoge_ptr spy_hoge;
    spy_hoge = realloc(orig, sizeof(*spy_hoge));
    if (!spy_hoge) {
        free(orig);
        return NULL;
    }
    memcpy(&spy_hoge->orig, orig, sizeof(spy_hoge->orig));
    memset(spy_hoge->call_count, 0, sizeof(spy_hoge->call_count));
    spy_hoge->spy.self_destroy = spy_destroy;
    spy_hoge->spy.inc_x = spy_inc_x;
    spy_hoge->spy.inc_10 = spy_inc_10;
    spy_hoge->spy.ge_x = spy_ge_x;

    return spy_hoge;
}
```

# Spy の実装(cont'd)

それぞれの spy メソッドは、各API呼び出しのカウントをインクリメントさせた上で、
original のメソッドを呼び出すようにします。

```C:spy_hoge.c
static void
spy_destroy(struct hoge_struct *self)
{
    if (!self) {
        return;
    }
    spy_hoge_ptr spy_hoge = (spy_hoge_ptr)self;
    for (int idx = 0; idx < API_NUM; idx++) {
        printf("idx:%s, count:%d\n",
                spy_hoge_id2str(idx), spy_hoge->call_count[idx]);
    }
    // you fall into infinite loop unless you use an original function.
    spy_hoge->spy.self_destroy = spy_hoge->orig.self_destroy;
    hoge_destroy(spy_hoge);
}

static void
spy_inc_x(struct hoge_struct *self, int x)
{
    spy_hoge_ptr spy_hoge = (spy_hoge_ptr)self;
    spy_hoge->call_count[API_INC_X]++;
    return hoge_inc(&spy_hoge->orig, x);
}

static void
spy_inc_10(struct hoge_struct *self)
{
    spy_hoge_ptr spy_hoge = (spy_hoge_ptr)self;
    spy_hoge->call_count[API_INC_10]++;
    return hoge_inc10(&spy_hoge->orig);
}

static bool
spy_ge_x(struct hoge_struct *self, int x)
{
    spy_hoge_ptr spy_hoge = (spy_hoge_ptr)self;
    spy_hoge->call_count[API_GE]++;
    return hoge_ge(&spy_hoge->orig, x);
}
```

# Spy の実装(cont'd)

spy メソッドの実装の他に、テスト用に各APIカウントを取得するAPIを実装します。

```C:spy_hoge.c
int
spy_hoge_count_inc_x(void *spy)
{
    spy_hoge_ptr spy_hoge = (spy_hoge_ptr)spy;
    return spy_hoge->call_count[API_INC_X];
}

int
spy_hoge_count_inc_10(void *spy)
{
    spy_hoge_ptr spy_hoge = (spy_hoge_ptr)spy;
    return spy_hoge->call_count[API_INC_10];
}

int
spy_hoge_count_ge(void *spy)
{
    spy_hoge_ptr spy_hoge = (spy_hoge_ptr)spy;
    return spy_hoge->call_count[API_GE];
}
```

# Spy テストの実装

ここまでで、Spy実装を使って hoge モジュールをテストすることができます。
まずは、GoogleTestヘッダファイルなど、関連ヘッダファイルをインクルードします。

```cpp:Test_hoge_spy.cpp
#include "gtest/gtest.h"

// include the api headers for "hoge"
#include "hoge.h"

// include spy headers for "hoge"
#include "spy_hoge.h"
```

# Spy テストの実装(cont'd)

Spy テスト用のクラスを定義します。
SetUp()において、spy_hoge_new()で spy 用の hoge クラスを生成します。

```cpp:Test_hoge_spy.cpp
// Test Class

class Test_hoge_spy: public ::testing::Test {
protected:
    virtual void SetUp() {
        hoge = spy_hoge_new();
    }
    virtual void TearDown() {
        hoge_destroy(hoge);
    }

    void *hoge;
};
```

# Spy テストの実装(cont'd)

spy 用の hoge クラスを生成した後は、
original の


```cpp:Test_hoge_spy.cpp
TEST_F(Test_hoge_spy, inc_x)
{
    for (int i = 0; i < 10; i++) {
        hoge_inc(hoge, 1);
    }
    ASSERT_TRUE(hoge_ge(hoge, 10));
    ASSERT_FALSE(hoge_ge(hoge, 11));

    ASSERT_EQ(10, spy_hoge_count_inc_x(hoge));
    ASSERT_EQ(0, spy_hoge_count_inc_10(hoge));
    ASSERT_EQ(2, spy_hoge_count_ge(hoge));
}

TEST_F(Test_hoge_spy, inc_10)
{
    hoge_inc10(hoge);
    ASSERT_TRUE(hoge_ge(hoge, 10));
    ASSERT_FALSE(hoge_ge(hoge, 11));

    ASSERT_EQ(0, spy_hoge_count_inc_x(hoge));
    ASSERT_EQ(1, spy_hoge_count_inc_10(hoge));
    ASSERT_EQ(2, spy_hoge_count_ge(hoge));
}
```

# Mocking
